# ## Cloning WebMap with non hosted Layer

In [1]:
from arcgis.gis import GIS, Item
from arcgis.mapping import WebMap

In [2]:
source = GIS(profile='esoapp_unetwork')
target = GIS(profile='esoapp2_unetwork')

In [3]:
web_map_item_id = '1e1cef78b96d4a43aeb748808f700dd0'
web_map = source.content.get(web_map_item_id)
web_map

<Item title:"WebMap_with_FeatureLayer" type:Web Map owner:unetwork>

In [4]:
source_map_layers = WebMap(web_map).layers
source_map_layers

[{
   "id": "cities_7104",
   "layerType": "ArcGISFeatureLayer",
   "url": "https://esoapp.gis.lt/hserver/rest/services/Hosted/cities/FeatureServer/0",
   "visibility": true,
   "opacity": 1,
   "mode": 1,
   "title": "cities",
   "itemId": "098d57031ff64ca9bd4e8e79221110c8",
   "layerDefinition": {
     "drawingInfo": {
       "renderer": {
         "type": "simple",
         "symbol": {
           "color": [
             255,
             0,
             0,
             255
           ],
           "outline": {
             "color": [
               110,
               110,
               110,
               255
             ],
             "width": 0.4,
             "type": "esriSLS",
             "style": "esriSLSSolid"
           },
           "type": "esriSFS",
           "style": "esriSFSSolid"
         }
       }
     }
   },
   "popupInfo": {
     "title": "polygons: {created_user}",
     "fieldInfos": [
       {
         "fieldName": "created_date",
         "label": "created

### Search for non hosted layers source Map content

In [5]:
source_map_layer = next(l for l in source_map_layers if not 'services/Hosted' in l.url)
source_map_layer_id = source_map_layer.itemId
print(f"{'Map Service Layer:':35}{source_map_layer.title}\n{'-'*80}")
print(f"{' '*2}{'item id:':20}{source_map_layer_id}")
print(f"{' '*2}individual layer urls:")
for l in source_map_layer.layers:
    print(f"{' '*4}{source_map_layer.url}/{l.id}")

Map Service Layer:                 BendriDuomenys
--------------------------------------------------------------------------------
  item id:            72722457f63544fe8fac6aea81f64896
  individual layer urls:
    https://esoapp.gis.lt/hserver/rest/services/Dujos/BendriDuomenys/MapServer/10
    https://esoapp.gis.lt/hserver/rest/services/Dujos/BendriDuomenys/MapServer/30
    https://esoapp.gis.lt/hserver/rest/services/Dujos/BendriDuomenys/MapServer/20


### Before copying WebMap, move non-hosted services separately

In [6]:
serach_filter = f"title:{source_map_layer.title} AND owner:{target.users.me.username}"
print(f"Search layers in target portal using query: {serach_filter}")
target_items = target.content.search(query=f"title:{source_map_layer.title} AND owner:{target.users.me.username}",  
                                        item_type="Map Image Layer")
print("Search result:")
target_map_layer = target_items[0]
target_map_layer_id = target_map_layer.id
print(f"{'Map Service Layer:':35}{target_map_layer.title}\n{'-'*80}")
print(f"{' '*2}{'item id:':20}{target_map_layer_id}")
print(f"{' '*2}individual layer urls:")
for flyr in target_map_layer.layers:
    print(f"{' '*4}{flyr.url}")

Search layers in target portal using query: title:BendriDuomenys AND owner:unetwork
Search result:
Map Service Layer:                 BendriDuomenys
--------------------------------------------------------------------------------
  item id:            518b0f866cc34dadbca6aab7db3e35a6
  individual layer urls:
    https://esoapp2.gis.lt/hserver/rest/services/Dujos/BendriDuomenys/MapServer/10
    https://esoapp2.gis.lt/hserver/rest/services/Dujos/BendriDuomenys/MapServer/30
    https://esoapp2.gis.lt/hserver/rest/services/Dujos/BendriDuomenys/MapServer/20


### Create mapping dictionary between source & target environments

In [7]:
item_mapping = {source_map_layer_id : target_map_layer_id}
item_mapping

{'72722457f63544fe8fac6aea81f64896': '518b0f866cc34dadbca6aab7db3e35a6'}

### Ready to copy WebMap using mapping dictionary

In [8]:
folder_name = 'DevEU24'
folder = next((f for f in target.users.me.folders if f['title'] == folder_name), None)
if not folder:
    folder = target.content.create_folder(folder=folder_name)
folder

{'username': 'unetwork',
 'id': '649b7ebd7ae142c5bc8d26c7aa9f3c88',
 'title': 'DevEU24',
 'created': 1727676075935}

In [9]:
try:
    cloned_items = target.content.clone_items(items=[web_map],
                                              use_org_basemap=False,
                                              folder=folder['title'],
                                              item_mapping=item_mapping)
except Exception as e:
    print("Failed to clone.")
    print(str(e))

In [10]:
cloned_items

[<Item title:"WebMap_with_FeatureLayer" type:Web Map owner:unetwork>]

In [21]:
cloned_web_map_item = next(item for item in cloned_items if item.type == 'Web Map')
cloned_web_map = WebMap(cloned_web_map_item)
cloned_web_map.layers
for l in cloned_web_map.layers:
    print(f"{'Layer:':10}{l.title}\n{'-'*80}")
    print(f"{' '*2}{'item id:':5}{l.id};{' '*2}{'url:':5}{l.url}")

Layer:    cities
--------------------------------------------------------------------------------
  item id:cities_7104;  url: https://esoapp2.gis.lt/hserver/rest/services/Hosted/cities/FeatureServer/0
Layer:    BendriDuomenys
--------------------------------------------------------------------------------
  item id:BendriDuomenys_7551;  url: https://esoapp2.gis.lt/hserver/rest/services/Dujos/BendriDuomenys/MapServer
